### Task 4: Optimize Portfolio Based on Forecast

Construct an efficient frontier using expected returns (TSLA from forecast; BND, SPY from historical averages) and the historical covariance matrix. Identify and mark the maximum Sharpe and minimum volatility portfolios.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data_manager import DataManager
from src.task4_data import Task4DataBuilder
from src.task4_optimizer import Task4Optimizer
from src.forecasting_analysis import ForecastAnalyzer
from models.forecasting_models import ARIMAForecaster
from src.model_selection import arima_order_grid_search

plt.style.use('seaborn-v0_8')
%matplotlib inline


### 1) Load prices for TSLA, BND, SPY


In [ ]:
START_DATE = '2015-07-01'
END_DATE = '2025-07-31'
TICKERS = ['TSLA', 'BND', 'SPY']

# Load from yfinance via DataManager
dm = DataManager(data_source='yfinance')
raw = dm.fetch_data(TICKERS, start_date=START_DATE, end_date=END_DATE, frequency='1d')
prices = pd.DataFrame({
    'TSLA': raw['TSLA']['Adj Close'],
    'BND': raw['BND']['Adj Close'],
    'SPY': raw['SPY']['Adj Close'],
}).dropna()
prices.tail()


### 2) Build expected returns and covariance

- TSLA expected return from ARIMA forecast (using 6-month horizon)
- BND and SPY expected returns from historical average daily return annualized
- Covariance from historical daily returns (annualized)


In [ ]:
# Fit a concise ARIMA on full TSLA history and forecast 6 months
order, _ = arima_order_grid_search(prices['TSLA'], p_values=range(0,4), d_values=range(0,2), q_values=range(0,4))
arima = ARIMAForecaster(order=order).fit(prices['TSLA'])

analyzer = ForecastAnalyzer(history=prices['TSLA'])
analyzer.attach_model('ARIMA', arima)
fc_6m = analyzer.forecast(horizon_days=126, include_intervals=True, ci_alpha=0.1)

# Build task 4 inputs
data_builder = Task4DataBuilder(prices)
inputs = data_builder.build_expected_inputs(tsla_forecast_prices=fc_6m.mean, method='compounded')

inputs.mu, inputs.cov.head()


### 3) Efficient Frontier and Key Portfolios


In [ ]:
opt = Task4Optimizer(risk_free_rate=0.02)
frontier = opt.efficient_frontier(inputs.daily_returns, num_portfolios=1500)
keys = opt.key_portfolios(inputs.daily_returns)

# Plot Efficient Frontier
fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(frontier.risks, frontier.returns, c=np.array(frontier.returns)/np.maximum(np.array(frontier.risks),1e-8), cmap='viridis', s=10)
ax.set_xlabel('Volatility (Annualized)')
ax.set_ylabel('Expected Return (Annualized)')
ax.set_title('Efficient Frontier (TSLA, BND, SPY)')

# Mark key portfolios
ax.scatter(keys.max_sharpe['volatility'], keys.max_sharpe['expected_return'], marker='*', color='red', s=200, label='Max Sharpe')
ax.scatter(keys.min_volatility['volatility'], keys.min_volatility['expected_return'], marker='X', color='blue', s=120, label='Min Volatility')
ax.legend();


### 4) Recommend a Portfolio

- If prioritizing risk-adjusted returns: choose the Maximum Sharpe portfolio.
- If prioritizing stability: choose the Minimum Volatility portfolio.

Weights and metrics for both are shown below; select based on the investor’s profile.


In [ ]:
weights_df = pd.concat([keys.max_sharpe_weights, keys.min_volatility_weights], axis=1)
weights_df.columns = ['MaxSharpe', 'MinVol']

summary = pd.DataFrame({
    'Portfolio': ['MaxSharpe', 'MinVol'],
    'ExpectedReturn': [keys.max_sharpe['expected_return'], keys.min_volatility['expected_return']],
    'Volatility': [keys.max_sharpe['volatility'], keys.min_volatility['volatility']],
    'Sharpe': [keys.max_sharpe['sharpe_ratio'], keys.min_volatility.get('sharpe_ratio', np.nan)],
})
weights_df, summary
